In [3]:
from spacy.tokens import DocBin
import spacy
import json
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin
import random

In [2]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [5]:
camp = load_data("data/camp.json")
def convert_data(data):
    final = []
    for item in data:
        text = item["text"]
        ents = item["spans"]
        final_ents = []
        for ent in ents:
            start, end, label = ent["start"], ent["end"], ent["label"]
            final_ents.append((start, end, label))
        ents = {"entities": final_ents}
        final.append([text, ents])
    return (final)
camp = convert_data(camp)
random.shuffle(camp)
camp_train = camp[0:500]
camp_valid = camp[501:1000]
write_data("data/camp_train.json", camp_train)
write_data("data/camp_valid.json", camp_valid)
print (camp_train[0])

['In charge of Gestapo headquarters In 1942 Gestapo took over the building of the state archives, "Arkivet" in Kristiansand.  The local Gestapo commander was Rudolf Kerner, and it was he who determined who should be heard. During the war was about 3,500 Norwegians from Sørlandet arrested on of more than four days. Over 400 were tortured by the German security police and Gestapo, and 162 died in concentration camps or were executed. One of the tortured was resistance fighter Louis Hogganvik. "Horror house" as the nickname was of Arkivet, is today the only surviving Gestapo headquarters in  Norway, and appears almost authentic. Kerner was in charge of several  German security police actions at Sørlandet in World War II together with the Norwegian Statspolitiet .', {'entities': [(344, 350, 'CAMP'), (665, 671, 'CAMP')]}]


In [24]:

nlp = spacy.blank("en") 
def create_training(TRAIN_DATA):
    db = DocBin() # create a DocBin object
    for text, annot in tqdm(TRAIN_DATA): # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        for start, end, label in annot["entities"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents # label the text with the ents
        db.add(doc)
    return (db)

In [25]:
camp_train = create_training(camp_train)
camp_train.to_disk("./data/camp_train.spacy")

camp_valid = create_training(camp_valid)
camp_valid.to_disk("./data/camp_valid.spacy")

100%|██████████| 499/499 [00:00<00:00, 1122.63it/s]
